## Get bugs information from `.csv` files

The following part extracts information from the `.csv` files dumped during running test suites on DBMSs.

In [31]:
import pandas as pd

bugs_df = pd.read_csv("output/mysql_bugs.csv")
logs_df = pd.read_csv("output/mysql_logs.csv")

print(bugs_df.info())
print(logs_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9270 entries, 0 to 9269
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       9270 non-null   int64 
 1   DBMS_NAME        9270 non-null   object
 2   TESTFILE_INDEX   9270 non-null   int64 
 3   TESTFILE_PATH    9270 non-null   object
 4   TESTCASE_INDEX   9270 non-null   int64 
 5   SQL              9270 non-null   object
 6   CASE_TYPE        9270 non-null   object
 7   EXPECTED_RESULT  8890 non-null   object
 8   ACTUAL_RESULT    8921 non-null   object
 9   EXEC_TIME        9270 non-null   int64 
 10  DATE             9270 non-null   object
 11  LOGS_INDEX       9270 non-null   int64 
dtypes: int64(5), object(7)
memory usage: 869.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LOGS    1118 non-null  

In [35]:
from random import randint

id = 2
bug_row = bugs_df.loc[id]
log_row = logs_df.loc[int(bug_row.LOGS_INDEX)]
print(logs_df.LOGS)
dbms_name = 'mysql'  # TODO need add a dbms_name field in the dataframe
test_suite = 'sqlite_tests'  # TODO same as above


0                                                     NaN
1       CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGE...
2       CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGE...
3       CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGE...
4       CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGE...
                              ...                        
1117    CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGE...
1118    CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGE...
1119    CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGE...
1120    CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGE...
1121    CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGE...
Name: LOGS, Length: 1122, dtype: object


## Reproduce an error testcase

The following code picks a bug record in the `.csv` file and run on the selected db. Maybe running using the testrunner.

In [36]:
from src import testrunner

if dbms_name == 'sqlite':
    r = testrunner.SQLiteRunner()
elif dbms_name == 'duckdb':
    r = testrunner.DuckDBRunner()
elif dbms_name == 'cockroachdb':
    r = testrunner.CockroachDBRunner()
elif dbms_name == 'mysql':
    r = testrunner.MySQLRunner()
else:
    exit("Not implement yet")
    
r.connect("tempdb")

print(bug_row)
print(log_row)
print(log_row.LOGS)
r.executemany_stmt(log_row.LOGS)

print("The Fault SQL is %s" % bug_row.SQL)
print("The Expected Result is:")
print(bug_row.EXPECTED_RESULT)
print("The Actual Result is:")
result = r.execute_query(bug_row.SQL)
for row in result:
    for item in row:
        print(item)






Unnamed: 0                                                         1
DBMS_NAME                                                      mysql
TESTFILE_INDEX                                                     0
TESTFILE_PATH                              sqlite_tests/select1.test
TESTCASE_INDEX                                                     1
SQL                INSERT INTO t1(e,c,b,d,a) VALUES(103,102,100,1...
CASE_TYPE                                                  Statement
EXPECTED_RESULT                                                 True
ACTUAL_RESULT                                                   True
EXEC_TIME                                                        827
DATE                                                  22-10-29-16:39
LOGS_INDEX                                                         1
Name: 1, dtype: object
LOGS    CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGE...
Name: 1, dtype: object
CREATE TABLE t1(a INTEGER, b INTEGER, c INTEGER, d INTEGER, e INTEGE